#IPO API Demo
This file will demonstrate access to the IPO Prophet api using the graphql interface.

In [ ]:
# Most kernels will not have gql so we need to install it
!pip install "gql[all]"

In [ ]:
# Import the packages
import pandas as pd
import requests
import json
import os
import time
import logging
from datetime import datetime, timedelta
import asyncio
import aiohttp
import csv
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from pprint import pprint

from dateutil.parser import parse

Update the username and password field with your credentials.

In [ ]:
# setup our variables
graphqlEndpoint = "https://api.ipoprophet.com/graphql"
gql_username = "<your_username>"
gql_password = "<your_password>"

We use cookie tokens for authentication so we will use a CookieJar to setup our graphql client to store and send the authentication token.

In [ ]:
# Setup GQL for use below
jar = aiohttp.CookieJar()
transport = AIOHTTPTransport(url=graphqlEndpoint, client_session_args={'cookie_jar': jar})
client = Client(transport=transport, fetch_schema_from_transport=False)


Next we need to execute the login mutation to authenticate ourselves.

In [ ]:
# before doing anything we need to log in
loginQuery = gql(
    """
    mutation login($email: String!, $password: String!) {
      login(email: $email, password: $password) {
        sessionId
        token
        sid
        wmid
      }
    }
    """
)
loginResults = await client.execute_async(loginQuery, variable_values= { "email": gql_username, "password": gql_password})
print(loginResults)

Once we have sucessfully authenticated we can get and subscribe to the information we want. The fields available can be found in the schema file at the end of this document.

In [ ]:
# Get Deals
async def getAllDeals():
  getDeals = gql(
      """
      query GetDeals {
        getDeals {
          symbol
          listingDate
        }
      }
      """
  )
  return await client.execute_async(getDeals)

When subscribing to deal information you will get a type value that will indicate if this is a newly created deal, and updated deal, or a removed deal.

In [ ]:
# Subscribe to deals
async def subscribeToDeals():
  subscribeDeals = gql(
      """
      subscription DealUpdates {
        dealUpdates {
          type
          deal {
            id
            symbol
            companyName
            exchange
            sharePrice
            numberOfShares
          }
        }
      }
      """
  )
  return await client.execute_async(subscribeDeals)

In [ ]:
# Subscribe to sell signals
async def subscribeToSellSignals():
  subscribeSell = gql(
      """
      subscription SellSignal {
        sellSignal {
          symbol
          time
          price
        }
      }
      """
  )
  return await client.execute_async(subscribeSell)

In [ ]:
# Subscribe to bull signal
async def subscribeToIndication():
  subscribeIndication = gql(
      """
      subscription IndicatedOpen {
        indicatedOpen {
          symbol
          pairedShares
          isBullish
          price
        }
      }
      """
  )
  return await client.execute_async(subscribeIndication)

# Graphql Schema
Below is the graphql schema that you can use in programs like Postman to get intellisense for your queries, subscriptions, and mutations.

```
"""The payload used to check if a bull signal should be triggered"""
input CheckForBullConditionsInput {
  """The indicated price of the symbol"""
  indicatedPrice: Float!

  """The current paired shares"""
  pairedShares: Float!

  """The symbol to check for a bullish condition"""
  symbol: String!
}

"""The payload to create a new deal into the IPO Prophet system"""
input CreateDealInput {
  """The company name"""
  companyName: String!

  """The external id for the deal"""
  dealId: String!

  """The exchange the deal will be listed on"""
  exchange: String!

  """The industry the company operates in"""
  industry: String!

  """The date of the IPO"""
  listingDate: DateTimeISO!

  """The number of shares being offered"""
  numberOfShares: Int

  """The proposed share price for the deal"""
  sharePrice: Float

  """The symbol for the deal"""
  symbol: String!
}

"""
A date-time string at UTC, such as 2007-12-03T10:15:30Z, compliant with the `date-time` format outlined in section 5.6 of the RFC 3339 profile of the ISO 8601 standard for representation of dates and times using the Gregorian calendar.This scalar is serialized to a string in ISO 8601 format and parsed from a string in ISO 8601 format.
"""
scalar DateTimeISO

"""Information about the IPO Prophet Deal"""
type Deal {
  """The price of a sell signal, if a sell signal is triggered"""
  absorption_sell_price: Float

  """The time of a sell signal, if a sell signal is triggered"""
  absorption_sell_time: DateTimeISO

  """The close price of the deal, set after IPO day"""
  close_price: Float

  """The company name for the deal"""
  companyName: String!
  dailyBars: [HistoricDailyOHLC!]!

  """The daily high price of the deal, set after IPO day"""
  daily_high: Float

  """The daily low price of the deal, set after IPO day"""
  daily_low: Float

  """The external id for the deal"""
  dealId: String!

  """The exchange the deal will be listed on"""
  exchange: String!

  """The internal id for the deal"""
  id: Float!

  """The last price indicated before opening"""
  indicatedOpenPrice: Float

  """The last number of pair share indicated before opening"""
  indicatedPairedShares: Int

  """The industry of the company"""
  industry: String
  ipoBars: [OHLC!]!

  """If a position should be taken on the deal"""
  isBullish: Boolean!

  """The date of the IPO"""
  listingDate: DateTimeISO

  """The price of a sell signal, if a sell signal is triggered"""
  momentum_sell_price: Float

  """The time of a sell signal, if a sell signal is triggered"""
  momentum_sell_time: DateTimeISO

  """The number of shares being offered for the deal"""
  numberOfShares: Int

  """The open price of the deal, set after IPO day"""
  open_price: Float

  """
  The sale price of the position based on the sell signals and daily close price
  """
  prophetExitPrice: Float

  """The proposed share price for the deal"""
  sharePrice: Float

  """The symbol for the deal"""
  symbol: String!

  """The total traded volume during the trading day"""
  totalVolume: Int
}

"""The deal subscription payload"""
type DealMessage {
  """The deal information"""
  deal: Deal!

  """The type of update made"""
  type: String!
}

"""Daily OLHC post IPO"""
type HistoricDailyOHLC {
  """The daily close"""
  close: Float!

  """The daily high"""
  high: Float!

  """Record id for the database entry"""
  id: Float!

  """The daily low"""
  low: Float!

  """The daily open price"""
  open: Float!

  """The symbol id for the equity"""
  symbol: String!

  """The day for the bar"""
  time: DateTimeISO!

  """The timestamp for the bar"""
  timestamp: Float!

  """The number of trades for the day"""
  trade_count: Float!

  """The value of trades for the day"""
  value_traded: Float!

  """The daily volume"""
  volume: Int!
}

"""The payload for uploading historical records"""
input HistoricIpoRecordInput {
  c: Float!
  h: Float!
  l: Float!
  o: Float!
  t: DateTimeISO!
  tradeCount: Int!
  v: Int!
  valueTraded: Float!
  vwap: Float!
}

type IndexFund {
  """The date of the return information"""
  date: DateTimeISO!

  """The Renaissance IPO price"""
  ipo: Float!

  """The normalized return on the IPO starting 2013-10-16"""
  ipo_normalized: Float!

  """The IPO Prophet value, starting a $1,000,000 entry"""
  ipo_prophet: Float!

  """The normalized return on IPO Prophet starting 2013-10-16"""
  ipo_prophet_normalized: Float!

  """The SPX price"""
  spx: Float!

  """The normalized return on the SPX starting 2013-10-16"""
  spx_normalized: Float!
}

type IndicatedOpen {
  """An indicator if a position should be taken"""
  isBullish: Boolean!

  """The current number of paired shares"""
  pairedShares: Int!

  """The current expected opening price"""
  price: Float!

  """The symbol for the indication"""
  symbol: String!
}

"""
The `JSON` scalar type represents JSON values as specified by [ECMA-404](http://www.ecma-international.org/publications/files/ECMA-ST/ECMA-404.pdf).
"""
scalar JSON

type LoginOutputType {
  sessionId: String!
  sid: String
  token: String!
  wmid: String
}

type Mutation {
  """Log into the IPO Prophet server"""
  login(email: String!, password: String!): LoginOutputType!

  """Log out of the IPO Prophet server"""
  logout: Boolean!
}

"""A news article"""
type News {
  """The content of the news article, in lexicaljs format"""
  body: String!

  """The date the article was published"""
  createdAt: DateTimeISO

  """The internal id of the news article"""
  id: Float!

  """The date the article was last updated"""
  modifiedAt: DateTimeISO

  """The title for the news article"""
  title: String!
}

"""The payload of a news article"""
input NewsInput {
  """The lexicaljs formatted content of the article"""
  body: String!

  """The title of the article"""
  title: String!
}

"""Open High Low Close 1-minute bar for a given equity"""
type OHLC {
  """The close price for the minute bar"""
  close: Float!

  """The high price for the minute bar"""
  high: Float!

  """Record id for the database entry"""
  id: Float!

  """The low price for the minute bar"""
  low: Float!

  """The open price for the minute bar"""
  open: Float!

  """The symbol id for the equity"""
  symbol: String!

  """The time for the bar"""
  time: DateTimeISO!

  """The timestamp for the bar"""
  timestamp: Float!

  """The number of trades for the minute bar"""
  trade_count: Float!

  """The value of trades for the minute bar"""
  value_traded: Float!

  """The volume traded for the minute bar"""
  volume: Int!
}

type Qmsymbol {
  datatype: String
  exchange: String
  symbol: String
}

type Qmtopic {
  name: String
}

type Qmtopics {
  qmsymbol: Qmsymbol
  qmtopic: Qmtopic
}

type Query {
  """Gets a deal with a specific Deal Id"""
  getDeal(
    """The deal id to search"""
    dealId: String!
  ): Deal!

  """Gets a deal by it's ticker symbol"""
  getDealBySymbol(
    """The symbol to search for"""
    symbol: String!
  ): Deal!

  """Get all deals in the system, ordered by listing date"""
  getDeals(
    """Either all deals, or only ones IPO prophet made a call on"""
    bullOnly: Boolean

    """The number of deals to skip"""
    skip: Int

    """The number of deals to take, limit 10,000"""
    take: Int
  ): [Deal!]!

  """Gets all deals on a specific date"""
  getDealsByDate(
    """The date to search for"""
    date: DateTimeISO!
  ): [Deal!]!

  """A list of daily bars per symbol"""
  getHistoricDailyBars(
    """The symbol to get the daily bars for"""
    symbol: String!
  ): [HistoricDailyOHLC!]!

  """Gets the index fund information for normalized return comparison"""
  getIndexFund: [IndexFund!]!

  """A list of 1-minute bars per symbol"""
  getIpoBars(
    """The symbol to get the 1-minute bars for"""
    symbol: String!
  ): [OHLC!]!
  getIpoCalendar: [TFilteredIPOs!]!
  getIpoNews: [QuoteMediaNewsitem!]!
  getIpoNewsArticle(newsid: String!): String!

  """The latest indicated open received"""
  getLatestIndicatedOpen(
    """The symbol to get the latest indicated open for"""
    symbol: String!
  ): IndicatedOpen

  """The sell signals for a current symbol"""
  getLatestSellSignal(
    """The symbol to get the sell signals for"""
    symbol: String!
  ): [SellSignal!]
  getNews: [News!]!
  getUser(id: Float!): User!
  getUsers(limit: Int, userRole: UserRole!): [User!]!

  symbolSearch(searchString: String!): [Quote!]!

  """Gets all deals for today"""
  todaysDeals: [Deal!]!
}

type Quote {
  dispSecIndFlag: Boolean
  exchDisp: String
  exchange: String
  index: String
  industry: String
  industryDisp: String
  isYahooFinance: Boolean
  longname: String
  quoteType: String
  score: Float
  sector: String
  sectorDisp: String
  shortname: String
  symbol: String
  typeDisp: String
}

type QuoteMediaNewsitem {
  datetime: String
  headline: String
  lang: String
  newsid: Float
  permalink: String
  qmtopics: Qmtopics
  source: String
}

"""An indication that you should exist 50% of your current position"""
type SellSignal {
  """The internal id of the signal"""
  id: Float!

  """The price the sell signal recommends you exit at"""
  price: Float!

  """The symbol the sell signal was trigger for"""
  symbol: String!

  """The time the signal was triggered"""
  time: DateTimeISO!
}

type Subscription {
  """A subscription to changes in deals on IPO Prophet"""
  dealUpdates: DealMessage!

  """
  A subscription to indicate if a position should be entered prior to the deal opening
  """
  indicatedOpen: IndicatedOpen!

  """Subscribe to the IPO 1-minute bars currently being watched"""
  listenForBar: OHLC!

  """A subscription to know if an exist condition is triggered"""
  sellSignal: SellSignal!
  subscribeToNews: News!

  """Subscribe to the IPO 1-minute bars for today"""
  todaysBars: [SubscriptionData!]!
  token: String!
}

type SubscriptionData {
  """All the 1-minute bars for a symbol"""
  bars: JSON!

  """The symbol of the subscription"""
  symbol: String!
}

type TFilteredIPOs {
  companyName: String!
  expected: String!
  offerAmount: String!
  priceRange: String!
  sharesOffered: Float!
  symbol: String!
  underwriter: String!
}

input UpdateUserInput {
  address: String
  avatar: String
  city: String
  country: String
  fName: String
  lName: String
  mName: String
  password: String
  phone: String
  removeAvatar: Boolean
  state: String
  status: UserStatus
  userRole: UserRole
  zip: String
}

type User {
  address: String
  avatar: String
  city: String
  country: String
  createdAt: DateTimeISO!
  email: String!
  fName: String
  id: Float!
  lName: String
  mName: String
  modifiedAt: DateTimeISO
  phone: String
  proUser: Boolean!
  publisher: Boolean
  qmuser: String
  qmwmid: String
  state: String
  status: String!
  userRole: String!
  username: String
  zip: String
}

"""The permission role of a user"""
enum UserRole {
  ADMIN
  USER
}

"""The account status of a user"""
enum UserStatus {
  Active
  Banned
  Deleted
  Disabled
  Inactive
}

input updateDealInput {
  """The company name"""
  companyName: String

  """The external id for the deal"""
  dealId: String

  """The exchange the deal will be listed on"""
  exchange: String

  """The industry the company operates in"""
  industry: String

  """The date of the IPO"""
  listingDate: DateTimeISO

  """The number of shares being offered"""
  numberOfShares: Int

  """The proposed share price for the deal"""
  sharePrice: Float

  """The symbol for the deal"""
  symbol: String
}
```

